## Proyecto de ML con IA  
Vamos a predecir el riesgo cardiovascular de una persona en función de una imagen. Para ello, entrenaremos un modelo con datos tabulares y unas features y le pasaremos la imagen de una persona a un LLM vía API. Le pediremos que nos extraiga las features que le digamos y las introduciremos en nuestro modelo.  

Los datos los sacamos de [aquí](https://archive.ics.uci.edu/dataset/2/adult)

In [ ]:
# Cambia las rutas a los archivos si es necesario
import pandas as pd
df_train = pd.read_csv("data_train.csv")
df_test = pd.read_csv("data_test.csv")
x_train = df_train.drop(columns='riesgo_cardiovascular')
y_train = df_train.loc[:,'riesgo_cardiovascular']
x_test = df_test.drop(columns='riesgo_cardiovascular')
y_test = df_test.loc[:,'riesgo_cardiovascular']

In [2]:
# Las clases están bien balanceadas
df_train.loc[:,'riesgo_cardiovascular'].value_counts()

riesgo_cardiovascular
2    584
0    473
1    393
Name: count, dtype: int64

In [11]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Vamos a lanzar tres modelos con hiperparametros al azar y ver cual da mejor resultado,
# en funcion de ese resultado vamos a usar un gridsearch para optimizar esos hiperparametros

accuracy_models = dict()
models_name = ['random_forest', 'xgboost', 'lgbm']
models = [RandomForestClassifier(n_estimators=50, max_depth=3),\
              XGBClassifier(n_estimators = 50, max_depth=3)]

for model, name in zip(models, models_name):
    model.fit(x_train,y_train)
    y_pred = model.predict(x_test)
    accuracy_models[name] = accuracy_score(y_test, y_pred)
accuracy_models

{'random_forest': 0.9812646370023419, 'xgboost': 0.990632318501171}

In [4]:
# Los resultados son excelentes, vamos a usar un gridsearch con XGBoost por hacerlo,
# pero no hace falta porque la accuracy es muy buena ya
from sklearn.model_selection import GridSearchCV

# Define the hyperparameter grid
param_grid = {
    'n_estimators':[50,75,100],
    'max_depth': [3, 5, 7]
    
}

# Create the XGBoost model object
xgb_model = XGBClassifier()

# Create the GridSearchCV object
grid_search = GridSearchCV(xgb_model, param_grid, cv=5, scoring='accuracy')

# Fit the GridSearchCV object to the training data
grid_search.fit(x_train, y_train)

# Print the best set of hyperparameters and the corresponding score
print("Best set of hyperparameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

best_model = grid_search.best_estimator_

Best set of hyperparameters:  {'max_depth': 3, 'n_estimators': 50}
Best score:  0.9799999999999999


In [5]:
# Nuestro best model es un XGBoost con n_est = 50, max_depth = 3
accuracy_score(y_test,best_model.predict(x_test))

0.990632318501171

In [6]:
# Nos lo guardamos en un pickle
import pickle
with open('best_model.pkl','wb') as file:
    pickle.dump(best_model,file)

In [8]:
# Este es el formato que necesitamos para predecir,
# un array 2D con 1 float, 1 entero y tres binarios
best_model.predict([[25,52,1,1,0]])

array([1])

In [ ]:
# Asegurate de tener un .env en la carpeta del notebook con tu clave de la api de gemini
# GEMINI_API_KEY='TU_CLAVE'
from dotenv import load_dotenv
load_dotenv()

True

In [34]:
# Vamos ahora con la parte de llamar a una API, introducir una imagen y que nos devuelva las features que queremos
from google.genai import types
from google import genai

client = genai.Client()
prompt = '''
Eres un asistente virtual especializado en el reconocimiento de características físicas de una persona. A partir de la imagen de una
persona extraerás las siguientes features:

1. Indice de masa corporal (IMC): tipo float
2. Edad: tipo int
3. Sexo: 0 hombre y 1 mujer
4. Ojeras: 0 si no tiene ojeras y 1 si sí que las tiene
5. Tabaquismo: 0 si crees que esa persona fuma y 1 si crees que esa persona fuma

Tu respuesta será únicamente una lista con una lista bidimensional con esa información y una buena
justificacion de por que has has asignado esos parametros a esa persona. Max 300 tokens
Por ejemplo:

Ejemplo de tu output: [[[22.3, 25, 0, 1 ,0]], "Esa pesona tiene 25 años porque (explicacion...)"]
'''

with open('images/image1.jpg', 'rb') as f:
    image_bytes = f.read()

response = client.models.generate_content(
model='gemini-2.5-flash',
contents=[
    types.Part.from_bytes(
    data=image_bytes,
    mime_type='image/jpeg',
    ),
    prompt
]
)

print(response.text)

[[[22.5, 30, 0, 0 ,1]], "Esta persona tiene un IMC estimado de 22.5 por su complexión atlética y saludable, sin aparentar sobrepeso ni delgadez extrema. Su edad parece ser de unos 30 años debido a su apariencia juvenil y la ausencia de signos visibles de envejecimiento. Es hombre (0) por sus rasgos faciales masculinos y la presencia de barba. No se pueden determinar ojeras (0) ya que lleva gafas de sol que cubren completamente esa zona. Se asume que no fuma (1) al no haber ninguna evidencia visual de tabaquismo en la imagen."]


In [37]:
# Ahora devolvemos ese string como lista
import json
json.loads(response.text)[0]

[[22.5, 30, 0, 0, 1]]

# Ejecuta esta celda

In [21]:
# Lo tenemos todo listo para pasarselo al modelo y que nos haga la predicción

from dotenv import load_dotenv
load_dotenv()

import pickle
with open('best_model.pkl','rb') as file: # Cambia la ruta si es necesario
    best_model = pickle.load(file)

from google.genai import types
from google import genai

client = genai.Client()
prompt = '''
Eres un asistente virtual especializado en el reconocimiento de características físicas de una persona. A partir de la imagen de una
persona extraerás las siguientes features:

1. Indice de masa corporal (IMC): tipo float
2. Edad: tipo int
3. Sexo: 0 hombre y 1 mujer
4. Ojeras: 0 si no tiene ojeras y 1 si sí que las tiene
5. Tabaquismo: 0 si crees que esa persona fuma y 1 si crees que esa persona fuma

Tu respuesta será únicamente una lista con una lista bidimensional con esa información y una buena
justificacion de por que has has asignado esos parametros a esa persona. Max 300 tokens
Por ejemplo:

Ejemplo de tu output: [[[22.3, 25, 0, 1 ,0]], "Esa pesona tiene 25 años porque (explicacion...)"]
'''

with open('../images/image1.jpg', 'rb') as f: # Añade aquí la ruta de la imagen que quieras analizar
    image_bytes = f.read()

response = client.models.generate_content(
model='gemini-2.5-flash',
contents=[
    types.Part.from_bytes(
    data=image_bytes,
    mime_type='image/jpeg',
    ),
    prompt
]
)


import json
output_api = json.loads(response.text)

def prediccion(output_api):
    if best_model.predict(output_api[0]) == 0:
        return f'Riesgo Cardiovascular bajo. {output_api[1]}'
    elif best_model.predict(output_api[0]) == 1:
        return f'Riesgo Cardiovascular medio. {output_api[1]}'
    else:
        return f'Riesgo Cardiovascular alto. {output_api[1]}'

prediccion(output_api)
    

'Riesgo Cardiovascular bajo. Su índice de masa corporal se estima en 21.0, ya que presenta una complexión delgada a media, con un cuerpo que no parece tener exceso de grasa. Se calcula una edad de 30 años, basándose en la ausencia de arrugas pronunciadas y el aspecto juvenil de su piel y cabello. El sexo es 0 (hombre) debido a sus características físicas masculinas evidentes, incluyendo vello facial. No se observan ojeras (0), ya que la zona debajo de sus ojos se ve clara y sin hinchazón ni decoloración. Finalmente, se clasifica como no fumador (1), ya que no hay signos visibles asociados al tabaquismo, como coloración en la piel o dientes, ni presencia de cigarrillos.'